In [1]:
import os 
import wandb
os.chdir( "/var/model" )

In [2]:
! ls -alh

total 14G
drwxrwxr-x  6 1001 1001 4.0K Nov 17 15:51 .
drwxr-xr-x  1 root root 4.0K Nov 17 15:36 ..
drwxr-xr-x  2 root root 4.0K Nov 11 03:01 .ipynb_checkpoints
-rw-rw-r--  1 1001 1001 6.9K Nov  5 17:36 LICENSE
-rw-rw-r--  1 1001 1001 6.1K Nov  5 17:36 README.md
-rw-rw-r--  1 1001 1001 4.7K Nov  5 17:36 USE_POLICY.md
-rw-r--r--  1 root root 9.3K Nov 11 02:12 code-llama-instruct-7b-peft.ipynb
-rw-r--r--  1 1001 1001  97K Nov 11 03:08 code-llama-instruct-7b.ipynb
-rw-rw-r--  1 1001 1001  646 Nov  5 17:36 config.json
-rw-rw-r--  1 1001 1001  116 Nov  5 17:36 generation_config.json
drwxr-xr-x  2 root root 4.0K Nov 17 15:47 merged
-rw-rw-r--  1 1001 1001 9.3G Nov  5 17:42 model-00001-of-00002.safetensors
-rw-rw-r--  1 1001 1001 3.3G Nov  5 17:40 model-00002-of-00002.safetensors
-rw-rw-r--  1 1001 1001  25K Nov  5 17:36 model.safetensors.index.json
-rw-rw-r--  1 1001 1001  24K Nov  5 17:36 pytorch_model.bin.index.json
-rw-rw-r--  1 1001 1001  411 Nov  5 17:36 special_tokens_map.json
-rw-r--r-

In [3]:
wandb.login()

wandb: Currently logged in as: ricardo-felipe-ruiz. Use `wandb login --relogin` to force relogin


True

In [4]:
%env WANDB_PROJECT=codellama-7b-instruct-hf-peft-fine-tuning

env: WANDB_PROJECT=codellama-7b-instruct-hf-peft-fine-tuning


In [5]:
import torch
from datasets import load_dataset
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments
)

from trl import SFTTrainer

# model_name = "meta-llama/Llama-2-7b-hf"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained( ".", add_eos_token=True, use_fast=True )
#Create a new token and add it to the tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

dataset = load_dataset( "timdettmers/openassistant-guanaco" )


/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


In [6]:

compute_dtype = getattr( torch, "float16" )
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)
print( bnb_config )
model = AutoModelForCausalLM.from_pretrained(
    ".", quantization_config=bnb_config, device_map={"": 0}
)

BitsAndBytesConfig {
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules= ["gate_proj", "down_proj", "up_proj"]
)

training_arguments = TrainingArguments(
        output_dir="./training-results",
        evaluation_strategy="steps",
        do_eval=True,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        per_device_eval_batch_size=4,
        log_level="debug",
        save_steps=100,
        logging_steps=100,
        learning_rate=4e-4,
        eval_steps=100,
        fp16=True,
        num_train_epochs=1,
        warmup_steps=100,
        lr_scheduler_type="cosine",
)

trainer = SFTTrainer(
        model=model,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=training_arguments,
)

Map:   0%|          | 0/518 [00:00<?, ? examples/s]

Using auto half precision backend


In [ ]:

trainer.train()

#stop reporting to wandb
wandb.finish()
# adapter = "./results/checkpoint-615/"

Currently training with a batch size of: 4
***** Running training *****
  Num examples = 9,846
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Training with DataParallel so batch size has been adjusted to: 4
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 8
  Total optimization steps = 307
  Number of trainable parameters = 23,199,744
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


You're using a CodeLlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
